# Coding Your Own Linear Regression Model

One task that you will almost certainly be required to do other data science courses (especially if you are a MIDS student) is to write up some of your statistical / machine learning models from scratch. This can be a very valuable exercise, as it ensures that you understand what is actually going on behind the scenes of the models you use ever day, and that you don't just think of them as "black boxes". 

To get a little practice doing this, today you will be coding up your own linear regression model! 

(If you are using this site but aren't actually in this class, you are welcome to skip this exercise if you'd like -- this is more about practicing Python in anticipation of the requirements of other courses than developing your applied data science skills.) 

There are, broadly speaking, two approaches you can take to coding up your own model: 

1. you can write the model by defining a new function, or 
2. you can write the model by defining a new class with associated methods (making a model that works the way a model works in `scikit-learn`). 

Whether you do 1 or 2 is very much a matter of choice and style. Approach one, for example, is more consistent with what is called a *functional* style of programming, while approach two is more consistent with an *object-oriented* style of programming. Python can readily support both approaches, so either would work fine. 

In these exercises, however, I will ask you to use approach number 2 as this *tends* to be the more difficult approach, and so practicing approach 2 will be extra useful in preparing you for other classes (HA! Pun...). In particular, our goal is to implement a linear regression model that has the same "initialize-fit-predict-score" API (application programming interface -- a fancy name for the methods a class supports) as `scikit-learn` models. That means your model should be able to do all of the following:

1. **Initialize** a new model. 
2. **Fit** a linear model when given a numpy vector (`y`) and a numpy matrix (`X`) with the syntax `my_model.fit(X, y)`. 
3. **Predict** values when given a new `numpy` matrix (`X_test`) with the syntax `my_model.predict(X_test)`. 
4. Return the **model coefficients** through the property `my_model.coefficients` (not quite what is used in `sklearn`, but let's use that interface). 

Also, bear in mind that throughout these exercises, we'll be working in `numpy` instead of `pandas`, just as we do in `scikit-learn`. So assume that before using your model, your user has already converted their data from `pandas` into `numpy` arrays. 

**(1)** Define a new Class called `MyLinearModel` with methods for `__init__`, `fit`, `predict`, and an attribute for `coefficients`. For now, we don't need any initialization *arguments*, just an `__init__` function. 

As you get your code outline going, start by just having each method `pass`:

```python
def my_method(self):
    pass
```

This will allow your methods to run without errors (they just don't do anything). Then we can double back to each method to get them working one by one.

In [ ]:
import numpy as np

class MyLinearModel:
    def __init__(self):
        self.coefficients = None

    def fit(self, X, y):
        pass

    def predict(self, X):
        pass

**(2)** Now define your `fit` method. This is the method that should actually run your linear regression. In case you've forgotten your linear algebra, remember that for linear regressions, $\beta = (X'X)^{-1}X'Y$, a fact you can see explained in detail on page four [here](https://www.stat.purdue.edu/~boli/stat512/lectures/topic3.pdf).

Note that once you have written the code to do a linear regression, you'll need to put your outputs (your coefficents) somewhere. I recommend making an attribute for your class where you can store your coefficients. 

(As a reminder: the normal multiply operator (`*`) in `numpy` implies scalar multiplication. Use `@` for matrix multiplication). 

**HINT:** Remember that linear regressions require a vector of 1s in the `X` matrix. As the package writer, you get to decide whether users are expected to provide a matrix `X` that already has a vector of 1s, or whether you expect the user to provide a matrix `X` that doesn't have a vector of 1s (in which case you will need to add a vector of 1s before you fit the model).

In [1]:
class MyLinearModel:
    def __init__(self):
        self.coefficients = None

    def fit(self, X, y):
        # Add a column of ones to X for the intercept term
        X_ = np.c_[np.ones(X.shape[0]), X]

        # Calculate the coefficients using the normal equation
        self.coefficients = np.linalg.inv(X_.T @ X_) @ X_.T @ y

**(3)** As you write code, it is good to test your code as you work. With that in mind, let's create some toy data. First, create a 100 x 2 matrix where each column is normally distributed. Then create a vector `y` that is a linear combination of those two columns **plus** a vector of normally distributed noise **and** a constant term. 

In other words, we want to create data where we *know* exactly what coefficients we should see so when we test our regression, we know if the results are accurate!

In [2]:
# Set seed for reproducibility
np.random.seed(42)

# Create a 100 x 2 matrix where each column is normally distributed
X = np.random.randn(100, 2)

# Create coefficients for the linear combination
true_coefficients = np.array([2, 3])

# Create a vector y that is a linear combination of X plus noise and a constant term
noise = np.random.randn(100)
y = X @ true_coefficients + 1 + noise

**(4)** Now test whether you `fit` method generates the correct coefficients. Remember the choice you made in Question 2 about whether your package expects the users' `X` matrix to include a vector of 1s when you test!

In [3]:
# Create an instance of MyLinearModel
my_model = MyLinearModel()

# Fit the model using the toy data
my_model.fit(X, y)

# Display the coefficients obtained by the model
print("Model Coefficients:", my_model.coefficients)

Model Coefficients: [1.09279338 2.19072034 2.8278579 ]


**(5)** Now let's make the statisticians proud, and in addition to storing the coefficients, let's store the standard errors for our estimated coefficients as another attribute. Recall that the simplest method of calculating the variance covariance matrix for $\beta$ is using the formula $\sigma^2 (X'X)^{-1}$, where $\sigma^2$ is the variance of the error terms of your regression. The standard errors for your coefficient estimates will be the diagonal values of that matrix. See page six [here](https://www.stat.purdue.edu/~boli/stat512/lectures/topic3.pdf) for a full derivation. 

In [4]:
class MyLinearModel:
    def __init__(self):
        self.coefficients = None
        self.standard_errors = None

    def fit(self, X, y):
        # Add a column of ones to X for the intercept term
        X_ = np.c_[np.ones(X.shape[0]), X]

        # Calculate the coefficients using the normal equation
        self.coefficients = np.linalg.inv(X_.T @ X_) @ X_.T @ y

        # Calculate the variance of the error terms
        residuals = y - X_ @ self.coefficients
        residual_variance = np.var(residuals)

        # Calculate the standard errors for the coefficients
        var_cov_matrix = residual_variance * np.linalg.inv(X_.T @ X_)
        self.standard_errors = np.sqrt(np.diagonal(var_cov_matrix))

**(6)** Now let's also add an R-squared attribute to the model.

In [5]:
class MyLinearModel:
    def __init__(self):
        self.coefficients = None
        self.standard_errors = None
        self.r_squared = None

    def fit(self, X, y):
        # Add a column of ones to X for the intercept term
        X_ = np.c_[np.ones(X.shape[0]), X]

        # Calculate the coefficients using the normal equation
        self.coefficients = np.linalg.inv(X_.T @ X_) @ X_.T @ y

        # Calculate the variance of the error terms
        residuals = y - X_ @ self.coefficients
        residual_variance = np.var(residuals)

        # Calculate the standard errors for the coefficients
        var_cov_matrix = residual_variance * np.linalg.inv(X_.T @ X_)
        self.standard_errors = np.sqrt(np.diagonal(var_cov_matrix))

        # Calculate R-squared
        y_mean = np.mean(y)
        ssr = np.sum(residuals**2)
        sst = np.sum((y - y_mean) ** 2)
        self.r_squared = 1 - (ssr / sst)

**(7)** Now we'll go ahead and cheat a little. Use `statsmodels` to fit your model with your toy data to ensure your standard errors and r-squared are correct!

In [6]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [8]:
import statsmodels.api as sm


class MyLinearModel:
    def __init__(self):
        self.coefficients = None
        self.standard_errors = None
        self.r_squared = None

    def fit(self, X, y):
        # Add a column of ones to X for the intercept term
        X_ = np.c_[np.ones(X.shape[0]), X]

        # Calculate the coefficients using the normal equation
        self.coefficients = np.linalg.inv(X_.T @ X_) @ X_.T @ y

        # Calculate the variance of the error terms
        residuals = y - X_ @ self.coefficients
        residual_variance = np.var(residuals)

        # Calculate the standard errors for the coefficients
        var_cov_matrix = residual_variance * np.linalg.inv(X_.T @ X_)
        self.standard_errors = np.sqrt(np.diagonal(var_cov_matrix))

        # Calculate R-squared
        y_mean = np.mean(y)
        ssr = np.sum(residuals**2)
        sst = np.sum((y - y_mean) ** 2)
        self.r_squared = 1 - (ssr / sst)

        return self  # Return the instance for chaining methods


# Using statsmodels to fit the model
X_ = sm.add_constant(X)  # Add a constant term to X for the intercept
model_statsmodels = sm.OLS(y, X_).fit()

# Display results
print("MyLinearModel Coefficients:", MyLinearModel().fit(X, y).coefficients)
print("MyLinearModel Standard Errors:", MyLinearModel().fit(X, y).standard_errors)
print("MyLinearModel R-squared:", MyLinearModel().fit(X, y).r_squared)

print("\nStatsmodels Coefficients:", model_statsmodels.params)
print("Statsmodels Standard Errors:", model_statsmodels.bse)
print("Statsmodels R-squared:", model_statsmodels.rsquared)

MyLinearModel Coefficients: [1.09279338 2.19072034 2.8278579 ]
MyLinearModel Standard Errors: [0.1064008  0.12372145 0.10606538]
MyLinearModel R-squared: 0.9134939109532426

Statsmodels Coefficients: [1.09279338 2.19072034 2.8278579 ]
Statsmodels Standard Errors: [0.10803365 0.1256201  0.10769308]
Statsmodels R-squared: 0.9134939109532426


**(8)** Now implement `predict`! Then test it against your original `X` data -- do you get back something very close to your true `y`?

In [9]:
class MyLinearModel:
    def __init__(self):
        self.coefficients = None
        self.standard_errors = None
        self.r_squared = None

    def fit(self, X, y):
        # Add a column of ones to X for the intercept term
        X_ = np.c_[np.ones(X.shape[0]), X]

        # Calculate the coefficients using the normal equation
        self.coefficients = np.linalg.inv(X_.T @ X_) @ X_.T @ y

        # Calculate the variance of the error terms
        residuals = y - X_ @ self.coefficients
        residual_variance = np.var(residuals)

        # Calculate the standard errors for the coefficients
        var_cov_matrix = residual_variance * np.linalg.inv(X_.T @ X_)
        self.standard_errors = np.sqrt(np.diagonal(var_cov_matrix))

        # Calculate R-squared
        y_mean = np.mean(y)
        ssr = np.sum(residuals**2)
        sst = np.sum((y - y_mean) ** 2)
        self.r_squared = 1 - (ssr / sst)

        return self  # Return the instance for chaining methods

    def predict(self, X):
        # Add a column of ones to X for the intercept term
        X_ = np.c_[np.ones(X.shape[0]), X]

        # Predict using the learned coefficients
        y_pred = X_ @ self.coefficients
        return y_pred


# Using our MyLinearModel to fit the model
model = MyLinearModel().fit(X, y)

# Predict using the original X data
y_pred = model.predict(X)

# Display the original y and the predicted y
print("Original y:", y)
print("Predicted y:", y_pred)

Original y: [ 1.93642276  7.42525117  0.91233362  7.51453187  0.31106199 -2.26184969
 -3.74088092 -3.6229123   0.43212744 -1.20022776  3.82485915 -2.00362251
  1.19800408  0.47649815 -1.39162787  6.11239055 -2.97295245 -1.25425973
 -4.94664673 -0.98391425  5.30569657 -2.00187284 -3.43031642  1.63737327
 -4.07381575  1.58187169  1.54546489  4.77809462 -2.32137588  5.26875999
 -1.24569204 -4.58483123  6.73934357  3.2149781   1.93185688  6.97081993
  3.5971363  -1.58732838 -0.38471438 -3.92675186  0.8394732   2.28624099
 -1.61727106  4.68281276  0.27998548  3.76558883 -1.86203794 -4.82809038
  4.14086061  0.71144722 -4.35356141 -1.17439889  4.01173734  6.3285705
 -0.22760693 -3.40131813  9.77709791  0.81225063 -2.13163822  6.31607878
 -1.07302894 -0.45948992  5.50381372 -3.70435348 -0.56369238 -3.14342112
  1.92858117  2.38081337 -1.97273562 -0.93481794  6.81762142 -3.09692704
  5.02839796 -5.4250382   1.9533285   3.0024338   0.53577134 -1.15712618
  6.22284987  0.20174023  1.52540782  1.

**(9)** Finally, create the *option* of fitting the model with or without a constant term. In other words, create an option so that, if the user passes a numpy array *without* a constant term, your code will add a vector of 1s before fitting the model. As in `scikit-learn`, make this an option you set during initialization. 

In [11]:
class MyLinearModel:
    def __init__(self, fit_intercept=True):
        self.coefficients = None
        self.standard_errors = None
        self.r_squared = None
        self.fit_intercept = fit_intercept

    def fit(self, X, y):
        if self.fit_intercept:
            # Add a column of ones to X for the intercept term
            X_ = np.c_[np.ones(X.shape[0]), X]
        else:
            X_ = X

        # Calculate the coefficients using the normal equation
        self.coefficients = np.linalg.inv(X_.T @ X_) @ X_.T @ y

        # Calculate the variance of the error terms
        residuals = y - X_ @ self.coefficients
        residual_variance = np.var(residuals)

        # Calculate the standard errors for the coefficients
        var_cov_matrix = residual_variance * np.linalg.inv(X_.T @ X_)
        self.standard_errors = np.sqrt(np.diagonal(var_cov_matrix))

        # Calculate R-squared
        y_mean = np.mean(y)
        ssr = np.sum(residuals**2)
        sst = np.sum((y - y_mean) ** 2)
        self.r_squared = 1 - (ssr / sst)

        return self  # Return the instance for chaining methods

    def predict(self, X):
        if self.fit_intercept:
            # Add a column of ones to X for the intercept term
            X_ = np.c_[np.ones(X.shape[0]), X]
        else:
            X_ = X

        # Predict using the learned coefficients
        y_pred = X_ @ self.coefficients
        return y_pred